In [13]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
from langchain.retrievers.document_compressors import FlashrankRerank

In [14]:

load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [15]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


In [16]:
loader = DirectoryLoader('../data_test', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1332.37it/s]


In [17]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256,
    chunk_overlap=64,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
# pretty_print_docs(chunked)

In [18]:
cache_store = LocalFileStore("./mxbai_cache/")

In [19]:

model = AutoModel.from_pretrained('mixedbread-ai/mxbai-embed-large-v1', trust_remote_code=True) 

model_name = "mixedbread-ai/mxbai-embed-large-v1"
model_kwargs = {'device': 'cpu'}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

In [20]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [22]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace="mixedbread-ai/mxbai-embed-large-v1")

In [23]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 1min 23s, sys: 44 s, total: 2min 7s
Wall time: 26.4 s


In [24]:
llm = Ollama(model="llama3", temperature=0)
query = "How do I defeat Dreki?"

In [25]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

As you start to deplete the Dreki’s health (approximately two bars), it’ll introduce a slightly new attack into its arsenal that sees the creature leaping from afar, straight into a chomp attack. This attack is indicated by a yellow circle that will appear before it launches the attack.
Don’t be afraid to use your Resurrection Stone if you have one. There’s no checkpoint throughout this fight, so if you happen to die again before the fight is over, you’ll respawn with your stone.
Once the Dreki reaches half health, it’ll introduce another new attack to the battle, where it’ll launch an unlockable (red) attack that sees it rearing up into the air, as it begins to charge an electrical AoE attack. If you’re within the electrical circle after it finishes charging, you’ll be dealt damage and temporarily stunned, unable to move or block incoming attacks.
Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either

In [26]:
%%time
print(llm.invoke(query))

Dreki, the pesky dragon from the world of Skyrim!

To defeat Dreki, you'll need to be prepared for a challenging fight. Here are some tips to help you emerge victorious:

1. **Choose the right equipment**: Make sure you're wearing armor that can withstand fire and has a decent amount of health points. You may also want to consider equipping a shield with a high block value.
2. **Use your surroundings**: Dreki is a flying dragon, so try to use the environment to your advantage. Look for areas where you can trap him or limit his movement, such as narrow corridors or small caves.
3. **Keep moving**: Don't stay in one spot for too long, as Dreki's fire breath can be devastating. Keep moving and use your agility to dodge his attacks.
4. **Use your abilities wisely**: As a Dragonborn, you have access to various shouts and abilities that can help you take down Dreki. Use your elemental attacks (e.g., ice or fire) to chip away at his health, and consider using your shout abilities to stun or d

In [27]:
%%time
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either the Leviathan Axe or the Blades of Chaos to do extra, more impactful damage.
If you happen to escape the attack, just be sure you stay clear of the electrified spots on the ground; otherwise, you’ll also succumb to being electrified – you’ll need to mash circle to escape the hold of the electricity.
----------------------------------------------------------------------------------------------------
Document 2:

Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be blocked; however, this isn’t necessarily very efficient as it makes it quite a bit more challenging to deal damage.
Need some health? Try looking beside the crates that are found along the outskirts of the area.
-----------------------------------------------------------------------

In [28]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)

In [29]:
%%time
c2 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 2.52 s, sys: 591 ms, total: 3.11 s
Wall time: 11.4 s


In [30]:
print(c2["result"])

According to the context, here's how you can defeat Dreki:

1. While the Dreki is charging up its electric attack, use that brief moment to charge your Leviathan Axe or Blades of Chaos for extra damage.
2. If you escape the attack, stay clear of electrified spots on the ground and mash circle to escape the hold of electricity.
3. Once the Dreki reaches half health, it will start using unblockable attacks, so be prepared to dodge or parry them.
4. To defeat Dreki, try to stun it by blocking or parrying its attack just before it hits you (use Dauntless Shield for a parry). This will give you an opportunity to deal heavy damage and use your Runic Attack.
5. Unleash as much heavy damage as possible while the Dreki is stunned, including using your Runic Attack.

Remember to keep an eye out for electrified spots on the ground and be prepared to dodge or block attacks when the Dreki reaches half health. Good luck!


In [31]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0), retriever=compression_retriever, chain_type="refine"
)
c3 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


CPU times: user 3.21 s, sys: 1.54 s, total: 4.74 s
Wall time: 17.1 s


In [32]:
print(c3["result"])

 To defeat the Dreki, you need to avoid its electric attack by mashing the circle button to escape, then attacking it with your axe or blades. Be careful to avoid the electrified spots on the ground. You can also charge your axe or blades during the brief moment while Dreki is charging its electric attack to do extra damage. You may need to repeat this process multiple times to defeat Dreki. If you are using a shield, you can also parry the Dreki's saliva ball attack by blocking it just before it hits you, which will deal damage and temporarily stun the creature. However, it is not recommended to rely solely on blocking its attacks as this will make dealing damage inefficient. Additionally, you can replenish health from crates located outskirts of the area.


In [33]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", 
)
c4 = chain({"query": query})

CPU times: user 2.6 s, sys: 697 ms, total: 3.3 s
Wall time: 25.3 s


In [34]:
print(c4["result"])

Based on the refined context, here's an updated answer:

To defeat Dreki, follow these steps:

1. **Initial Attack**: As soon as the fight starts, be prepared for Dreki's initial saliva attack. You can either dodge it or block it with your shield (or parry it back into the creature if you're using the Dauntless Shield). If you choose to block or parry, take advantage of the brief stun effect to deal some damage.
2. **Runic Attack**: Once Dreki is stunned, rush in and unleash as much heavy damage as possible before the stun effect runs out. This includes using your Runic Attack early in the fight to get the cooldown timer going as soon as possible.
3. **Blockable Attacks**: Until Dreki reaches half health, all its attacks are blockable. While it's technically possible to block every attack, keep in mind that this might make dealing damage more challenging. Instead, focus on taking advantage of the brief moments when Dreki is stunned or vulnerable to deal significant damage.
4. **Health 

In [35]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", return_source_documents=True
)
c5 = chain({"query": query})

CPU times: user 3.26 s, sys: 1.5 s, total: 4.76 s
Wall time: 24.5 s


In [36]:
print(c5['source_documents'])

[Document(page_content='Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either the Leviathan Axe or the Blades of Chaos to do extra, more impactful damage.\nIf you happen to escape the attack, just be sure you stay clear of the electrified spots on the ground; otherwise, you’ll also succumb to being electrified – you’ll need to mash circle to escape the hold of the electricity.', metadata={'source': '../data_test/The Quest for Tyr.txt', 'relevance_score': 0.96820515}), Document(page_content='Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be blocked; however, this isn’t necessarily very efficient as it makes it quite a bit more challenging to deal damage.\nNeed some health? Try looking beside the crates that are found along the outskirts of the area.', metadata={'source': '../data_test/The Quest for Tyr.tx

In [37]:
print(c5["result"])

Based on the refined context, here's an updated answer:

To defeat Dreki, follow these steps:

1. **Initial Attack**: As soon as the fight starts, be prepared for Dreki's initial saliva attack. You can either dodge it or block it with your shield (or parry it back into the creature if you're using the Dauntless Shield). If you choose to block or parry, take advantage of the brief stun effect to deal some damage.
2. **Runic Attack**: Once Dreki is stunned, rush in and unleash as much heavy damage as possible before the stun effect runs out. This includes using your Runic Attack early in the fight to get the cooldown timer going as soon as possible.
3. **Blockable Attacks**: Until Dreki reaches half health, all its attacks are blockable. While it's technically possible to block every attack, keep in mind that this might make dealing damage more challenging. Instead, focus on taking advantage of the brief moments when Dreki is stunned or vulnerable to deal significant damage.
4. **Health 